In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import openmeteo_requests
import requests_cache
from retry_requests import retry

from modules.columnas_seleccionadas import COLUMNAS_SELECCIONADAS
from modules.tarifa_electrica import calcular_tarifa_electrica_general, cargos_por_anio, periodos

In [4]:
df = pd.read_csv('data/processed/dataset_final.csv')
df.columns

df.shape

(1185, 404)

In [5]:
df = df[COLUMNAS_SELECCIONADAS]
df['DIA'] = pd.to_datetime(df['DIA'], errors='coerce')
df['Anio'] = df['DIA'].dt.year
df['Mes'] = df['DIA'].dt.month
df['Dia'] = df['DIA'].dt.day

# INCLUSIÓN DÍA DE LA SEMANA

dias_semana = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miercoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}

df['Dia_semana'] = df['DIA'].dt.day_name().map(dias_semana)
print(df['Dia_semana'].unique()) 
# --------------------------

# INCLUSIÓN TEMPERATURA AMBIENTE
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# El cambio clave está en esta URL
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 32.5672,
    "longitude": -116.6251,
    "start_date": "2020-01-01",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)


hourly_dataframe["date_local"] = hourly_dataframe["date"].dt.tz_convert("America/Mexico_City")
hourly_dataframe["DIA"] = hourly_dataframe["date_local"].dt.date

temperaturas_diarias = (
    hourly_dataframe.groupby("DIA")["temperature_2m"]
    .mean()
    .reset_index()
    .rename(columns={"temperature_2m": "Temperatura_amb"})
)

df["DIA"] = pd.to_datetime(df["DIA"]).dt.date
temperaturas_diarias["DIA"] = pd.to_datetime(temperaturas_diarias["DIA"]).dt.date

df = df.merge(temperaturas_diarias, on="DIA", how="left")


# ------------------------------

#df.drop(columns=['DIA'], inplace=True)
print(df.shape)

['Miercoles' 'Jueves' 'Viernes' 'Sabado' 'Domingo' 'Lunes' 'Martes']
(1185, 36)


In [6]:
df.head()

,DIA,Servicios (Kw),KW Gral Planta,Planta (Kw),Agua Planta (Hl),Planta de agua (Hl),Frio (Kw),KW Trafo 10,Produccion (Hl),Totalizador Sistema Kaeser,...,KW Obrador Contratistas,Hl Producido Bodega,KW Cond 5. 6 y 9,KW Planta de Agua,Tot L3. L4 y Planta de CO2,Anio,Mes,Dia,Dia_semana,Temperatura_amb
0,2020-07-01,23848.0,59058.0,27637.27,10280.0,11241.40,14325.0,6046.25,2398.0,31871.0,...,41.0,43.000,614.0,43.0,0.0,2020,7,1,Miercoles,18.919001
1,2020-07-02,38033.0,131184.0,54409.81,13970.0,22107.77,23954.0,10108.13,25148.5,51079.0,...,-7603.0,7005.025,1073.0,-8533.0,0.0,2020,7,2,Jueves,21.044001
2,2020-07-03,42565.5,136078.0,65685.59,36300.0,46955.43,28268.0,9177.75,29365.0,62958.0,...,75.0,7463.250,1083.0,207.0,0.0,2020,7,3,Viernes,24.950249
3,2020-07-04,39650.0,139714.0,67098.54,40120.0,51124.18,24246.0,6717.25,32150.8,69435.0,...,83.0,12964.300,1031.0,224.0,0.0,2020,7,4,Sabado,25.531500
4,2020-07-05,45385.0,146862.0,70600.64,38940.0,49146.08,29885.0,9527.25,30793.9,66431.0,...,87.0,8908.975,1077.0,217.0,0.0,2020,7,5,Domingo,27.466917


In [7]:
df = calcular_tarifa_electrica_general(df, periodos, cargos_por_anio)

['Servicios (Kw)', 'KW Gral Planta', 'Planta (Kw)', 'KW Trafo 10', 'Kw Compresores Aire', 'Aire (Kw)', 'Elaboracion (Kw)', 'Pta Agua / Eflu (Kw)', 'KW Secador Kaeser', 'Bodega (Kw)', 'KW Trafo 11', 'KW Trafo 9', 'Efluentes (Kw)', 'KW Trafo 5', 'KW Enfluente Efl', 'KW Obrador Contratistas', 'KW Cond 5. 6 y 9', 'KW Planta de Agua']


In [8]:
df.columns

Index(['DIA', 'Servicios (Kw)', 'KW Gral Planta', 'Planta (Kw)',
       'Agua Planta (Hl)', 'Planta de agua (Hl)', 'Frio (Kw)', 'KW Trafo 10',
       'Produccion (Hl)', 'Totalizador Sistema Kaeser', 'Aire Producido (M3)',
       'Kw Compresores Aire', 'Hl de Mosto', 'Aire (Kw)', 'Elaboracion (Kw)',
       'Aire Planta (M3)', 'Temp Tq Intermedio', 'Pta Agua / Eflu (Kw)',
       'KW Secador Kaeser', 'Bodega (Kw)', 'KW Trafo 11', 'KW Trafo 9',
       'Efluentes (Kw)', 'KW Trafo 5', 'Agua Cond REC', 'KW Enfluente Efl',
       'KW Obrador Contratistas', 'Hl Producido Bodega', 'KW Cond 5. 6 y 9',
       'KW Planta de Agua', 'Tot L3. L4 y Planta de CO2', 'Anio', 'Mes', 'Dia',
       'Dia_semana', 'Temperatura_amb', 'Tarifa_electrica'],
      dtype='object')

In [9]:
def _get_estacion_hemisferio_norte(fecha):
    mes = fecha.month
    dia = fecha.day
    
    # Invierno: 21 de Dic - 19 de Mar
    if (mes == 12 and dia >= 21) or (mes == 1) or (mes == 2) or (mes == 3 and dia < 20):
        return "Invierno"
    # Primavera: 20 de Mar - 20 de Jun
    elif (mes == 3 and dia >= 20) or (mes == 4) or (mes == 5) or (mes == 6 and dia < 21):
        return "Primavera"
    # Verano: 21 de Jun - 22 de Sep
    elif (mes == 6 and dia >= 21) or (mes == 7) or (mes == 8) or (mes == 9 and dia < 23):
        return "Verano"
    # Otoño: 23 de Sep - 20 de Dic
    elif (mes == 9 and dia >= 23) or (mes == 10) or (mes == 11) or (mes == 12 and dia < 21):
        return "Otoño"
    else:
        return "Indefinido"

def agregar_columna_estacion(df, col_anio='Anio', col_mes='Mes', col_dia='Dia'):
    
    df_resultado = df.copy()
    
    columnas_fecha = {col_anio: 'year', col_mes: 'month', col_dia: 'day'}
    try:
        df_resultado['fecha_temp'] = pd.to_datetime(
            df_resultado[[col_anio, col_mes, col_dia]].rename(columns=columnas_fecha)
        )
    except Exception as e:
        print(f"Error al convertir columnas a fecha: {e}")
        print("Asegúrate de que las columnas 'anio', 'mes' y 'dia' existan y sean correctas.")
        return df
    
    df_resultado['estacion'] = df_resultado['fecha_temp'].apply(_get_estacion_hemisferio_norte)
    
    df_resultado = df_resultado.drop(columns=['fecha_temp'])
    
    return df_resultado


df = agregar_columna_estacion(df, col_anio='Anio', col_mes='Mes', col_dia='Dia')
df.columns

Index(['DIA', 'Servicios (Kw)', 'KW Gral Planta', 'Planta (Kw)',
       'Agua Planta (Hl)', 'Planta de agua (Hl)', 'Frio (Kw)', 'KW Trafo 10',
       'Produccion (Hl)', 'Totalizador Sistema Kaeser', 'Aire Producido (M3)',
       'Kw Compresores Aire', 'Hl de Mosto', 'Aire (Kw)', 'Elaboracion (Kw)',
       'Aire Planta (M3)', 'Temp Tq Intermedio', 'Pta Agua / Eflu (Kw)',
       'KW Secador Kaeser', 'Bodega (Kw)', 'KW Trafo 11', 'KW Trafo 9',
       'Efluentes (Kw)', 'KW Trafo 5', 'Agua Cond REC', 'KW Enfluente Efl',
       'KW Obrador Contratistas', 'Hl Producido Bodega', 'KW Cond 5. 6 y 9',
       'KW Planta de Agua', 'Tot L3. L4 y Planta de CO2', 'Anio', 'Mes', 'Dia',
       'Dia_semana', 'Temperatura_amb', 'Tarifa_electrica', 'estacion'],
      dtype='object')

In [10]:
df.head()

,DIA,Servicios (Kw),KW Gral Planta,Planta (Kw),Agua Planta (Hl),Planta de agua (Hl),Frio (Kw),KW Trafo 10,Produccion (Hl),Totalizador Sistema Kaeser,...,KW Cond 5. 6 y 9,KW Planta de Agua,Tot L3. L4 y Planta de CO2,Anio,Mes,Dia,Dia_semana,Temperatura_amb,Tarifa_electrica,estacion
0,2020-07-01,23848.0,59058.0,27637.27,10280.0,11241.40,14325.0,6046.25,2398.0,31871.0,...,614.0,43.0,0.0,2020,7,1,Miercoles,18.919001,1.033860e+08,Verano
1,2020-07-02,38033.0,131184.0,54409.81,13970.0,22107.77,23954.0,10108.13,25148.5,51079.0,...,1073.0,-8533.0,0.0,2020,7,2,Jueves,21.044001,1.915471e+08,Verano
2,2020-07-03,42565.5,136078.0,65685.59,36300.0,46955.43,28268.0,9177.75,29365.0,62958.0,...,1083.0,207.0,0.0,2020,7,3,Viernes,24.950249,2.261382e+08,Verano
3,2020-07-04,39650.0,139714.0,67098.54,40120.0,51124.18,24246.0,6717.25,32150.8,69435.0,...,1031.0,224.0,0.0,2020,7,4,Sabado,25.531500,2.275877e+08,Verano
4,2020-07-05,45385.0,146862.0,70600.64,38940.0,49146.08,29885.0,9527.25,30793.9,66431.0,...,1077.0,217.0,0.0,2020,7,5,Domingo,27.466917,2.414452e+08,Verano


In [11]:
df["Frio (Kw)"] = df["Frio (Kw)"].shift(-1)

In [12]:
import os

# --- 3. Guardar el archivo ---
nombre_carpeta = "data/processed"
nombre_archivo = "dataset_final.csv"
ruta_completa = os.path.join(nombre_carpeta, nombre_archivo)

# Crear carpeta si no existe
os.makedirs(nombre_carpeta, exist_ok=True)

try:
    # Guardar todas las filas menos la última
    df.iloc[:-1].to_csv(ruta_completa, index=False, encoding='utf-8-sig')
    print(f"\n✅ ¡Éxito! DataFrame guardado en: {ruta_completa}")
except Exception as e:
    print(f"\n❌ Error al guardar el archivo: {e}")



✅ ¡Éxito! DataFrame guardado en: data/processed\dataset_final.csv
